In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from keras.layers import LSTM,Dense
from keras.models import Sequential
from keras.utils import plot_model

In [ ]:
train  = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Tweets/train_tweets.csv')
test = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Tweets/test_tweets.csv')

In [ ]:
train.head()

,tweet_id,tweet_user_id,tweet_created_at_year,tweet_created_at_month,tweet_created_at_day,tweet_created_at_hour,tweet_hashtag_count,tweet_url_count,tweet_mention_count,tweet_has_attachment,tweet_attachment_class,tweet_language_id,tweet_topic_ids,virality
0,34698,10,2015,12,5,3,2.0,1.0,0.0,False,C,0,"['36', '36', '36', '36', '36', '36', '37', '37...",3
1,24644,4,2020,6,19,0,0.0,1.0,0.0,False,C,0,"['43', '78', '79', '80', '80', '89', '98', '99...",3
2,36321,54,2019,6,2,15,2.0,3.0,0.0,True,A,0,"['79', '80', '98', '98', '98', '99', '99', '10...",1
3,2629,42,2020,9,6,17,0.0,1.0,1.0,True,A,0,"['43', '79', '80', '98', '99', '99', '79', '80']",2
4,28169,32,2020,11,4,17,2.0,1.0,0.0,True,A,0,"['79', '80', '98', '99', '43', '89']",2


In [ ]:
train.tweet_attachment_class.value_counts()

A    17511
C    11825
B      289
Name: tweet_attachment_class, dtype: int64

In [ ]:
test.tweet_attachment_class.value_counts()

A    7505
C    5076
B     116
Name: tweet_attachment_class, dtype: int64

In [ ]:
test.tweet_attachment_class.map({'A':1,'B':2,'C':3})

0        1
1        3
2        3
3        3
4        1
        ..
12692    3
12693    3
12694    3
12695    1
12696    1
Name: tweet_attachment_class, Length: 12697, dtype: int64

In [ ]:
train.pop('tweet_topic_ids')
train.pop('tweet_attachment_class')
y = train.pop('virality')

test.pop('tweet_topic_ids')
test.pop('tweet_attachment_class')

0        A
1        C
2        C
3        C
4        A
        ..
12692    C
12693    C
12694    C
12695    A
12696    A
Name: tweet_attachment_class, Length: 12697, dtype: object

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20737, 11)
(8888, 11)
(20737,)
(8888,)


In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train['tweet_user_id'].to_numpy().reshape(-1,1),y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
print(clf.score(X_train['tweet_user_id'].to_numpy().reshape(-1,1),y_train))
print(clf.score(X_test['tweet_user_id'].to_numpy().reshape(-1,1),y_test))

0.6431981482374499
0.6364761476147615


#Users Data

In [ ]:
users = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Users/users.csv')

In [ ]:
users.head()

,user_id,user_like_count,user_followers_count,user_following_count,user_listed_on_count,user_has_location,user_tweet_count,user_has_url,user_verified,user_created_at_year,user_created_at_month
0,0,1164,48720,70469,5956,True,14122,True,0,2010,8
1,1,3914,85361,2171,5943,False,6957,False,0,2011,9
2,2,8292,200944,1416,8379,True,83485,True,0,2011,6
3,3,1770,15385,4572,1866,True,12265,True,0,2011,8
4,4,15311,459083,1021,7368,False,121193,False,0,2011,6


In [ ]:
users.shape

(52, 11)

In [ ]:
users.describe()

,user_id,user_like_count,user_followers_count,user_following_count,user_listed_on_count,user_tweet_count,user_verified,user_created_at_year,user_created_at_month
count,52.000000,52.000000,5.200000e+01,5.200000e+01,52.000000,52.000000,52.000000,52.000000,52.000000
mean,27.211538,4673.865385,2.452791e+05,4.214858e+04,6903.365385,39396.326923,0.019231,2011.250000,7.307692
std,16.363921,5714.506714,3.697964e+05,1.583784e+05,7815.721366,77111.642876,0.138675,1.311712,3.316852
min,0.000000,0.000000,3.632000e+03,0.000000e+00,276.000000,96.000000,0.000000,2008.000000,1.000000
25%,13.750000,1236.000000,2.271200e+04,9.720000e+02,1860.750000,6833.250000,0.000000,2011.000000,5.750000
50%,26.500000,2669.000000,8.727050e+04,2.675500e+03,4329.000000,12149.000000,0.000000,2011.000000,8.000000
75%,41.250000,6363.750000,2.981062e+05,1.189725e+04,8935.500000,29342.500000,0.000000,2012.000000,10.000000
max,55.000000,33091.000000,1.587783e+06,1.118149e+06,38838.000000,432467.000000,1.000000,2014.000000,12.000000


In [ ]:
users.isnull().sum().sum()

0

In [ ]:
users.dtypes

user_id                  int64
user_like_count          int64
user_followers_count     int64
user_following_count     int64
user_listed_on_count     int64
user_has_location         bool
user_tweet_count         int64
user_has_url              bool
user_verified            int64
user_created_at_year     int64
user_created_at_month    int64
dtype: object

In [ ]:
train.loc[0]

tweet_id                                                              34698
tweet_user_id                                                            10
tweet_created_at_year                                                  2015
tweet_created_at_month                                                   12
tweet_created_at_day                                                      5
tweet_created_at_hour                                                     3
tweet_hashtag_count                                                       2
tweet_url_count                                                           1
tweet_mention_count                                                       0
tweet_has_attachment                                                  False
tweet_attachment_class                                                    C
tweet_language_id                                                         0
tweet_topic_ids           ['36', '36', '36', '36', '36', '36', '37', '37...
virality    

In [ ]:
train = pd.concat([train,pd.DataFrame(columns=users.columns)])

In [ ]:
test = pd.concat([test,pd.DataFrame(columns=users.columns)])

In [ ]:
train.loc[0,'user_id':'user_created_at_month'] = users[users.user_id==train.tweet_user_id[0]].iloc[0]

In [ ]:
for i in range(train.shape[0]):
  train.loc[i,'user_id':'user_created_at_month'] = users[users.user_id==train.tweet_user_id[i]].iloc[0]

In [ ]:
for i in range(test.shape[0]):
  test.loc[i,'user_id':'user_created_at_month'] = users[users.user_id==test.tweet_user_id[i]].iloc[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20737, 22)
(8888, 22)
(20737,)
(8888,)


In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

1.0
0.6657290729072908


In [ ]:
clf2 = RandomForestClassifier(n_estimators=1200,max_depth=25, min_samples_leaf=1, min_samples_split=10)
clf2.fit(X_train,y_train)
print(clf2.score(X_train,y_train))
print(clf2.score(X_test,y_test))

0.8325698027679992
0.6750675067506751


In [ ]:
clf3 = RandomForestClassifier(n_estimators=1200,max_depth=25, min_samples_leaf=1, min_samples_split=10)
clf3.fit(train,y)
print(clf3.score(X_train,y_train))
print(clf3.score(X_test,y_test))

0.8364758643969716
0.8383213321332134


In [ ]:
sol4 = test.tweet_id.to_frame()
sol4['virality'] = clf3.predict(test)
sol4.astype(int).to_csv('sol8.csv',index=False)

In [ ]:
user_desc = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Users/user_vectorized_descriptions.csv')

In [ ]:
user_desc.head()

,user_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,...,feature_728,feature_729,feature_730,feature_731,feature_732,feature_733,feature_734,feature_735,feature_736,feature_737,feature_738,feature_739,feature_740,feature_741,feature_742,feature_743,feature_744,feature_745,feature_746,feature_747,feature_748,feature_749,feature_750,feature_751,feature_752,feature_753,feature_754,feature_755,feature_756,feature_757,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,0,0.132536,-0.137393,-0.064037,-0.118342,-0.130279,0.048067,-0.421301,-0.313038,0.047779,0.041972,-0.211500,0.157389,-0.119609,-0.167288,-0.183701,0.162600,-0.118144,-0.160549,0.206170,-0.349808,-0.180516,-0.075424,-0.228215,-0.227588,-0.206130,0.097065,-0.201940,0.113164,0.115008,-0.025116,0.063400,-0.129166,-0.154574,-0.219841,-0.185450,-0.099904,-0.084291,-0.044961,0.110390,...,0.083985,-0.157461,0.285158,-0.197924,-0.163785,0.535255,0.027747,-0.155363,0.146396,-0.090979,-0.170317,0.254166,0.260563,-0.071186,-0.140582,0.310176,0.083907,-0.034472,-0.172740,0.126395,-0.004203,-0.177539,0.038244,-0.188420,-0.080583,0.065391,0.265358,0.307018,-0.494297,-0.142920,0.238264,-0.315408,-0.159851,-0.003840,0.213492,0.002498,0.177574,-0.136515,-0.012882,0.017399
1,1,0.107849,-0.168418,0.027251,-0.075079,-0.084762,0.076149,-0.390708,-0.271934,0.007423,0.030401,-0.216736,0.183259,-0.069264,-0.236452,-0.209206,0.174043,-0.121529,-0.150529,0.228872,-0.336505,-0.204807,-0.152244,-0.307261,-0.216196,-0.265559,0.077822,-0.346440,0.154961,0.165459,-0.000246,0.065532,-0.173314,-0.191337,-0.143802,-0.223451,-0.067280,-0.124719,-0.160180,0.010054,...,0.036054,-0.140715,0.224058,-0.174127,-0.159510,0.531637,-0.003619,-0.117995,0.093102,-0.086952,-0.189147,0.209478,0.246669,-0.043450,-0.158822,0.295335,0.058998,-0.008168,-0.144616,0.219429,0.049639,-0.211484,0.026302,-0.199768,-0.131321,0.020595,0.304960,0.283139,-0.525245,-0.187449,0.232922,-0.314534,-0.177011,-0.041710,0.209785,-0.023427,0.158203,-0.143221,0.030484,0.081693
2,2,0.122312,-0.159376,-0.073417,-0.149442,-0.122684,-0.005277,-0.351233,-0.297342,-0.006010,0.083945,-0.243968,0.184267,-0.045257,-0.191175,-0.168322,0.190007,-0.150225,-0.191811,0.260278,-0.323330,-0.226146,-0.106863,-0.163877,-0.207189,-0.153667,0.090430,-0.265063,0.103507,0.147642,-0.003167,0.083048,-0.220785,-0.242494,-0.238759,-0.194130,-0.034603,0.002399,-0.173476,0.056946,...,0.003421,-0.165013,0.254066,-0.213777,-0.134803,0.554688,-0.024580,-0.159201,0.116502,-0.111342,-0.140976,0.216088,0.219368,-0.052936,-0.126136,0.385574,0.039820,-0.023451,-0.120135,0.191185,0.016503,-0.223201,0.051937,-0.162366,-0.111131,0.047493,0.281597,0.339442,-0.440569,-0.059945,0.173621,-0.292476,-0.185078,-0.026784,0.184902,0.009539,0.217004,-0.091951,0.025304,0.058501
3,3,0.160509,-0.137915,-0.002524,-0.034696,0.028126,0.056299,-0.365196,-0.259523,-0.037929,0.104135,-0.206807,0.194023,-0.105497,-0.277824,-0.154094,0.185838,-0.147508,-0.183590,0.282249,-0.251785,-0.132236,-0.152960,-0.293629,-0.165441,-0.207462,0.033447,-0.275356,0.135713,0.106392,-0.023706,0.049851,-0.122355,-0.158445,-0.189165,-0.210765,0.043706,-0.079914,-0.075443,-0.005512,...,0.034914,-0.239188,0.287563,-0.316668,-0.133856,0.503008,-0.039165,-0.090734,0.055525,-0.082432,-0.095903,0.214028,0.232646,-0.098938,-0.117663,0.290847,-0.023200,-0.031113,-0.156534,0.187953,0.039809,-0.193605,0.044424,-0.119147,0.001465,-0.090170,0.228475,0.299477,-0.412852,-0.191728,0.205752,-0.300688,-0.133753,0.002206,0.245214,-0.056659,0.152064,-0.180211,0.022327,0.014688
4,4,0.099192,-0.14080

In [ ]:
user_desc.shape

(52, 769)

In [ ]:
user_img = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Users/user_vectorized_profile_images.csv')

In [ ]:
user_img.head()

,user_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,...,feature_2008,feature_2009,feature_2010,feature_2011,feature_2012,feature_2013,feature_2014,feature_2015,feature_2016,feature_2017,feature_2018,feature_2019,feature_2020,feature_2021,feature_2022,feature_2023,feature_2024,feature_2025,feature_2026,feature_2027,feature_2028,feature_2029,feature_2030,feature_2031,feature_2032,feature_2033,feature_2034,feature_2035,feature_2036,feature_2037,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,0,0.449126,0.000000,0.000000,0.0,0.006348,0.000000,0.059772,0.026143,0.013903,0.000348,0.025731,0.464529,0.016236,0.000000,0.066158,0.000000,0.000000,0.119278,0.010585,0.010925,0.175525,0.034174,0.00000,0.044575,0.010829,0.167918,0.075417,0.525636,0.020431,0.054191,0.045793,0.075506,0.196820,0.001700,0.661213,0.047318,0.028807,0.024076,0.000000,...,0.00045,0.000000,0.000000,0.536466,0.000000,0.000000,0.083716,0.083041,0.027311,0.069096,0.000000,0.000000,0.000000,0.000000,0.000525,0.001897,0.000000,0.0,0.000000,0.000000,0.009984,0.000848,1.074829,0.242375,0.027740,0.000000,0.286132,0.011297,0.032700,0.027545,0.202886,0.000000,0.414033,0.000000,0.001876,0.000000,0.00000,0.000000,0.192305,0.000009
1,1,0.017050,0.009737,0.002633,0.0,0.000000,0.000000,1.184751,0.000000,0.000000,0.042436,0.000000,0.297861,0.007126,0.000000,0.078338,0.000000,0.000685,0.000000,0.316630,0.000000,0.242320,0.000000,0.08313,0.000000,0.000000,0.004858,0.027106,0.117184,0.010450,0.004800,0.026172,0.044867,0.369582,0.047539,0.182544,0.000467,0.151356,0.076805,0.010786,...,0.00000,0.000000,0.000000,0.436107,0.303672,0.191465,0.455144,0.050813,0.000000,0.000243,0.046793,0.100177,0.000000,0.489770,0.000000,0.549729,0.004078,0.0,0.034326,0.000000,0.026195,0.000000,0.002054,0.145270,0.091601,0.000000,0.000000,0.000000,0.002272,0.000485,0.111723,0.086082,0.000000,0.000000,0.412172,0.000000,0.00000,0.254945,0.536927,0.460923
2,2,0.000000,0.068666,0.000000,0.0,0.000000,0.085349,0.349153,0.000000,0.000000,0.000000,0.000000,0.520052,0.043143,0.000000,0.000000,0.022519,0.000000,0.000446,0.000000,0.305101,0.000000,0.171112,0.00000,0.000000,0.000000,0.000000,0.134256,0.315113,0.000000,0.000000,0.112964,0.027715,0.154152,0.016018,0.972479,0.009501,0.048328,0.059502,0.000000,...,0.00000,0.021245,0.038089,0.769890,0.001667,0.647853,0.026776,0.000000,0.152245,0.005618,0.000000,0.000000,0.154669,0.419150,0.058605,0.171305,0.088889,0.0,0.000000,0.010271,0.000000,0.005644,0.023498,0.035467,0.169990,0.060312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054352,0.000000,0.219981,0.00000,0.000000,0.150882,0.000000
3,3,0.000000,0.168951,0.118449,0.0,0.000000,0.000000,0.392767,0.000000,0.000000,0.000000,0.000000,0.377547,0.000000,0.000000,0.000000,0.000000,0.002089,0.166473,0.000000,0.000829,0.012438,0.025962,0.00000,0.193643,0.000000,0.000000,0.046941,0.024092,0.000000,0.249963,0.000000,0.097312,0.000000,0.000000,0.157172,0.180169,0.000000,0.001815,0.000000,...,0.00000,0.000000,0.000000,0.273602,0.000000,1.346873,0.082853,0.000000,0.000000,0.000000,0.000000,0.002634,0.000000,0.040582,0.000000,0.051374,0.000000,0.0,0.000000,0.008074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032581,0.016538,0.000000,0.168835,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.424234,0.000000
4,4,0.286990,0.299091,0.138534,0.0,0.000000,0.000000,0.360286,0.042764,0.014573,0.003912,0.000000,0.130987,0.145699,0.019894,0.324197,0.000486,0.000000,0.349089,0.010257,0.000000,0.000000,0.000000,0.00000,0.033734,0.123600,0

In [ ]:
user_img.shape

(52, 2049)

#Tweet Text

In [ ]:
train_text = pd.read_csv('/content/drive/MyDrive/Viral Tweets Prediction Challenge Dataset/Dataset/Tweets/train_tweets_vectorized_text.csv')

In [ ]:
train_text.shape

(29625, 769)

In [ ]:
train_text.head()

,tweet_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,...,feature_728,feature_729,feature_730,feature_731,feature_732,feature_733,feature_734,feature_735,feature_736,feature_737,feature_738,feature_739,feature_740,feature_741,feature_742,feature_743,feature_744,feature_745,feature_746,feature_747,feature_748,feature_749,feature_750,feature_751,feature_752,feature_753,feature_754,feature_755,feature_756,feature_757,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,0,0.125605,-0.136067,-0.121691,-0.160296,-0.074407,0.119014,-0.343523,-0.289790,-0.037007,0.120231,-0.245443,0.199461,-0.154236,-0.200109,-0.206436,0.270252,-0.142692,-0.102078,0.157226,-0.334515,-0.264958,-0.112983,-0.293211,-0.253694,-0.104198,0.056506,-0.231244,0.152571,0.206752,-0.150545,0.112063,-0.129411,-0.224150,-0.175330,-0.165828,-0.066047,-0.159027,0.009872,0.019299,...,0.039782,-0.174679,0.148821,-0.192575,-0.114211,0.496451,0.040274,-0.142680,0.169754,-0.075535,-0.117306,0.261488,0.240786,-0.150380,-0.080656,0.310319,0.042854,0.048131,-0.172710,0.135926,-0.043390,-0.208796,0.040137,-0.190645,-0.096934,-0.009036,0.284776,0.338148,-0.440536,-0.090837,0.215511,-0.330016,-0.143669,-0.017097,0.211852,0.009358,0.205395,-0.100113,0.013015,0.053247
1,1,0.064982,-0.116850,0.034871,-0.090357,-0.067459,0.030954,-0.361263,-0.294617,-0.077854,0.135007,-0.192705,0.252616,-0.135662,-0.201412,-0.183382,0.173640,-0.103182,-0.074723,0.234004,-0.283560,-0.120644,-0.063076,-0.248546,-0.224326,-0.176795,0.061400,-0.243843,0.226394,0.101096,-0.077593,0.057844,-0.086949,-0.239860,-0.303655,-0.223538,-0.041548,-0.162694,0.005842,0.053615,...,0.151876,-0.153876,0.272216,-0.265888,-0.124845,0.500886,-0.053478,-0.159796,0.102271,0.032116,-0.034348,0.292187,0.236578,-0.006660,-0.113676,0.249192,0.048188,-0.055551,-0.037698,0.148909,0.064823,-0.270230,0.003926,-0.207080,-0.062248,-0.056531,0.188629,0.366379,-0.511710,-0.025049,0.193301,-0.391395,-0.120417,-0.072493,0.188275,-0.084694,0.152518,-0.109684,0.034304,0.018237
2,4,0.051160,-0.076732,0.005174,-0.071699,-0.204004,0.034764,-0.320014,-0.231828,-0.121784,0.101362,-0.238145,0.173951,-0.102029,-0.181864,-0.214877,0.186110,-0.032114,-0.143620,0.175421,-0.260034,-0.103828,-0.122353,-0.316970,-0.289015,-0.215771,0.027695,-0.254362,0.160985,0.040491,-0.019251,0.156431,-0.089619,-0.204530,-0.155480,-0.207329,-0.044228,-0.094432,-0.054102,-0.069460,...,0.013487,-0.145385,0.251336,-0.243751,-0.218401,0.557650,-0.032927,-0.233999,0.107764,-0.054706,-0.151404,0.268172,0.148024,0.066601,-0.126532,0.235418,0.013908,0.107383,-0.114999,0.242328,0.062410,-0.122995,0.026454,-0.118704,-0.025266,0.015129,0.252958,0.273657,-0.522295,-0.049114,0.163904,-0.299526,-0.099811,-0.049208,0.170104,-0.125188,0.111381,-0.180564,-0.061082,0.142650
3,5,0.127061,-0.063152,0.010520,0.000385,-0.146983,-0.099981,-0.382142,-0.287832,-0.129653,0.056506,-0.180725,0.183492,-0.055121,-0.205312,-0.267817,0.152828,-0.026461,-0.150457,0.253863,-0.289086,-0.194721,-0.073793,-0.313012,-0.311311,-0.254014,-0.055075,-0.147885,0.179036,0.120235,-0.005531,0.080192,-0.229052,-0.137060,-0.281633,-0.225555,-0.010621,-0.105480,-0.135987,-0.034342,...,0.058002,-0.146499,0.203021,-0.242434,-0.169852,0.488979,0.018102,-0.178110,0.155078,-0.020591,-0.145490,0.284350,0.186599,0.037712,-0.143745,0.180907,-0.023579,0.016043,-0.097705,0.168376,0.072525,-0.184797,0.048695,-0.136387,-0.102923,-0.042237,0.235340,0.317434,-0.532257,-0.036280,0.168049,-0.355778,-0.150411,-0.067176,0.216000,-0.076183,0.158889,-0.057773,-0.079182,0.057414
4,8,0.144889,-0.084

In [ ]:
train_text.virality.value_counts(normalize=1)*100

1    46.015190
2    30.005063
3    15.682700
5     4.465823
4     3.831224
Name: virality, dtype: float64

In [ ]:
train_text['virality']=0

In [ ]:
train.head()

In [ ]:
for i in range(train_text.shape[0]):
  train_text.loc[i,'virality'] = y[train.tweet_id==train_text.tweet_id[i]].iloc[0]

In [ ]:
Y_text = train_text.pop('virality')

In [ ]:
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(train_text, Y_text, test_size=0.3)
print(X_train_text.shape)
print(X_test_text.shape)
print(y_train_text.shape)
print(y_test_text.shape)

(20737, 769)
(8888, 769)
(20737,)
(8888,)


In [ ]:
clf5 = RandomForestClassifier()
clf5.fit(X_train_text,y_train_text)
print(clf5.score(X_train_text,y_train_text))
print(clf5.score(X_test_text,y_test_text))

1.0
0.5435418541854186


#LSTM

In [ ]:
X_train_array = X_train_text.to_numpy().reshape(X_train_text.shape[0],X_train_text.shape[1],1)
X_test_array = X_test_text.to_numpy().reshape(X_test_text.shape[0],X_test_text.shape[1],1)

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=X_train_array.shape[1:]))
model.add(Dense(50,'relu'))
model.add(Dense(25,'relu'))
model.add(Dense(1,'sigmoid'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train_array, y_train_text.to_numpy(), epochs = 10, verbose=2,batch_size=256)

Epoch 1/10
82/82 - 193s - loss: -4.8624e+01 - accuracy: 0.4571
Epoch 2/10
82/82 - 192s - loss: -4.7353e+02 - accuracy: 0.4571
Epoch 3/10
82/82 - 192s - loss: -1.8288e+03 - accuracy: 0.4571
Epoch 4/10
82/82 - 191s - loss: -4.7966e+03 - accuracy: 0.4571
Epoch 5/10
82/82 - 193s - loss: -1.0015e+04 - accuracy: 0.4571
Epoch 6/10
82/82 - 191s - loss: -1.7950e+04 - accuracy: 0.4571
Epoch 7/10
82/82 - 192s - loss: -2.9315e+04 - accuracy: 0.4571
Epoch 8/10
82/82 - 191s - loss: -4.4333e+04 - accuracy: 0.4571
Epoch 9/10
82/82 - 192s - loss: -6.3555e+04 - accuracy: 0.4571
Epoch 10/10
82/82 - 193s - loss: -8.7663e+04 - accuracy: 0.4571


In [ ]:
preds = model.evaluate(X_train_array, y_train_text)
print ("Loss = " + str(preds[0]))
print ("Train Accuracy = " + str(preds[1]))

preds = model.evaluate(X_test_array, y_test_text)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

649/649 [==============================] - 98s 150ms/step - loss: -101714.5703 - accuracy: 0.4571
Loss = -101714.5703125
Train Accuracy = 0.4570574462413788
278/278 [==============================] - 46s 165ms/step - loss: -101645.1016 - accuracy: 0.4674
Loss = -101645.1015625
Test Accuracy = 0.46737173199653625
